In [4]:
#  Required Libraries
import pandas as pd
from datetime import datetime

#  File Paths
DATA_PATH = 'custom_data.csv'
TIMESTAMP_FILE = 'last_extraction.txt'

# ------------------------------------------------------------------------------
#  Section 1: Full Extraction
# ------------------------------------------------------------------------------

# Load the entire dataset
df_full = pd.read_csv(DATA_PATH, parse_dates=['transaction_date'])

# Display basic stats
print(" Full Extraction:")
print(f"Number of rows: {df_full.shape[0]}")
print(f"Number of columns: {df_full.shape[1]}")
print("Sample data:")
print(df_full.head())

# Print extraction message
print(f"\n Extracted {df_full.shape[0]} rows fully.\n")


# ------------------------------------------------------------------------------
#  Section 2: Incremental Extraction
# ------------------------------------------------------------------------------

# Step 1: Read the last extraction timestamp
try:
    with open(TIMESTAMP_FILE, 'r') as f:
        last_extraction_str = f.read().strip()
        last_extraction_time = datetime.strptime(last_extraction_str, "%Y-%m-%d %H:%M:%S")
        print(f" Last extraction time: {last_extraction_str}")
except FileNotFoundError:
    print(" No previous extraction found. Assuming first run.")
    last_extraction_time = datetime.min  # Extract all records

# Step 2: Perform incremental extraction
df_full['transaction_date'] = pd.to_datetime(df_full['transaction_date'])
df_incremental = df_full[df_full['transaction_date'] > last_extraction_time]

# Step 3: Display incremental results
print(f"\n Incremental Extraction:")
print(f"Found {df_incremental.shape[0]} new or updated records.")
print(df_incremental.head())

# ------------------------------------------------------------------------------
#  Section 3: Save New Timestamp
# ------------------------------------------------------------------------------

# Save current timestamp (assuming this is when the ETL ran)
current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

with open(TIMESTAMP_FILE, 'w') as f:
    f.write(current_timestamp)

print(f"\n Updated last extraction timestamp to: {current_timestamp}")

# ------------------------------------------------------------------------------
#  Section 4: Transform Full Data
# ------------------------------------------------------------------------------

# 1. Remove duplicates
df_full_cleaned = df_full.drop_duplicates()

# 2. Enrich with total_price column
df_full_cleaned['total_price'] = df_full_cleaned['price'] * df_full_cleaned['quantity']

# 3. Format date to YYYY-MM-DD
df_full_cleaned['transaction_date'] = pd.to_datetime(df_full_cleaned['transaction_date'])
df_full_cleaned['transaction_date'] = df_full_cleaned['transaction_date'].dt.strftime('%Y-%m-%d')

# Save to CSV
df_full_cleaned.to_csv('transformed_full.csv', index=False)
print(f"\n Transformed full dataset saved to transformed_full.csv with {df_full_cleaned.shape[0]} rows.")


# ------------------------------------------------------------------------------
#  Section 5: Transform Incremental Data
# ------------------------------------------------------------------------------

# 1. Remove duplicates
df_incremental_cleaned = df_incremental.drop_duplicates()

# 2. Enrich with total_price column
df_incremental_cleaned['total_price'] = df_incremental_cleaned['price'] * df_incremental_cleaned['quantity']

# 3. Format date to YYYY-MM-DD
df_incremental_cleaned['transaction_date'] = pd.to_datetime(df_incremental_cleaned['transaction_date'])
df_incremental_cleaned['transaction_date'] = df_incremental_cleaned['transaction_date'].dt.strftime('%Y-%m-%d')

# Save to CSV
df_incremental_cleaned.to_csv('transformed_incremental.csv', index=False)
print(f"\n Transformed incremental dataset saved to transformed_incremental.csv with {df_incremental_cleaned.shape[0]} rows.")






 Full Extraction:
Number of rows: 100
Number of columns: 6
Sample data:
  transaction_id customer_id     product    price  quantity  \
0          T0001       C6428  Headphones  1212.46         4   
1          T0002       C5981     Printer  1131.98         5   
2          T0003       C3602       Phone   541.59         4   
3          T0004       C7884  Headphones   538.00         2   
4          T0005       C4351  Headphones  1173.99         3   

     transaction_date  
0 2025-05-28 14:15:16  
1 2025-06-06 12:04:16  
2 2025-05-28 19:27:16  
3 2025-05-28 12:02:16  
4 2025-06-04 18:24:16  

 Extracted 100 rows fully.

 Last extraction time: 2025-06-13 10:52:56

 Incremental Extraction:
Found 0 new or updated records.
Empty DataFrame
Columns: [transaction_id, customer_id, product, price, quantity, transaction_date]
Index: []

 Updated last extraction timestamp to: 2025-06-13 10:53:25

 Transformed full dataset saved to transformed_full.csv with 100 rows.

 Transformed incremental dataset 